In [1]:
import os
from utils.dataset import SpectrogramsDataset
from utils.preprocessing import transform
from torch.utils.data import random_split
from torch import Generator

#split dataset deterministically
spec_dir = os.path.join('data', 'spec')
features_path = os.path.join('data', 'features.csv')
dataset = SpectrogramsDataset(spec_dir, features_path, transform, target='danceability')
trainset, valset, testset = random_split(dataset, [0.02, 0.02, 0.96], Generator().manual_seed(42)) #TODO 0.8,0.1,0.1
len(trainset), len(valset), len(testset)

(19, 19, 910)

In [2]:
from torch.utils.data import DataLoader

#configure batches
kwargs = {
    'batch_size': 4, #TODO tune with Neptune
    'num_workers': 2,
    'pin_memory': True,
    'drop_last': True}
train_loader = DataLoader(trainset, shuffle=True, persistent_workers=True, **kwargs)
val_loader = DataLoader(valset, persistent_workers=True, **kwargs)
test_loader = DataLoader(testset, **kwargs)
next(iter(train_loader))[0].shape

torch.Size([4, 3, 224, 224])

In [3]:
#TODO adjust this cell

import torch
from utils.models import DanceabilityModel
from torch import nn, optim

#configure training hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DanceabilityModel().to(device) #TODO tune with Neptune
criterion = nn.MSELoss() #TODO tune with Neptune
optimizer = optim.Adam(model.parameters(), lr=1e-3) #TODO tune with Neptune
device

'cpu'

In [4]:
from copy import deepcopy
from contextlib import suppress
import numpy as np

train_losses = []
val_losses = []
best_params = None
for epoch in range(999):

    #train
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device)) #estimates loss of batch with current parameters
        optimizer.zero_grad() #resets gradient to avoid accumulation
        loss.backward() #computes gradient of loss w.r.t. parameters
        optimizer.step() #updates parameters to reduce loss
        running_loss += loss.item()
    train_losses.append(running_loss/len(train_loader))
    print(f'[epoch {epoch:03}] train loss: {train_losses[-1]:.4f}, ', end='')

    #evaluate
    model.eval()
    running_loss = 0
    for inputs, labels in val_loader:
        outputs = model(inputs.to(device))
        running_loss += criterion(outputs, labels.to(device)).item()
    val_losses.append(running_loss/len(val_loader))

    #back up best parameters
    end = '\n'
    if val_losses[-1] <= min(val_losses):
        best_params = deepcopy(model.state_dict())
        end = ' (new best)\n'
    print(f'val loss: {val_losses[-1]:.4f}', end=end)

    #stop early
    patience = 3
    with suppress(IndexError):
        if all(np.array(val_losses[-patience:]) > val_losses[-patience-1]):
            break

[epoch 000] train loss: 0.0579, val loss: 0.0635 (new best)
[epoch 001] train loss: 0.0546, val loss: 0.0231 (new best)
[epoch 002] train loss: 0.0401, val loss: 0.0140 (new best)
[epoch 003] train loss: 0.0419, val loss: 0.0120 (new best)
[epoch 004] train loss: 0.0433, val loss: 0.0119 (new best)
[epoch 005] train loss: 0.0379, val loss: 0.0120
[epoch 006] train loss: 0.0381, val loss: 0.0119 (new best)
[epoch 007] train loss: 0.0296, val loss: 0.0133
[epoch 008] train loss: 0.0409, val loss: 0.0144
[epoch 009] train loss: 0.0420, val loss: 0.0138


# TODO

* Check:
    * [Image classification example](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
    * [Tabular regression example](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)
    * [My VAE](https://github.com/octodoge/dssc-pml/blob/main/code/vae_pyro.ipynb)
    * [Marta's classifier](https://github.com/15Max/MusicalGenreRecognition/blob/main/project.ipynb)